In [75]:
import pandas as pd
import numpy as np
from datetime import datetime

In [76]:
df_listing = pd.read_csv(r'C:\Users\matte\OneDrive\Desktop\GitHub\data\Airbnb\listings.csv')
df_reviews = pd.read_csv(r'C:\Users\matte\OneDrive\Desktop\GitHub\data\Airbnb\reviews.csv')

In [77]:
display(df_reviews.head())
display(df_listing.head())

,listing_id,date
0,6400,2010-04-19
1,6400,2011-04-16
2,6400,2012-04-22
3,6400,2014-04-11
4,6400,2014-04-14


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6400,Rental unit in Milan · ★4.89 · 3 bedrooms · 1 bed · 3.5 baths,13822,Francesca,NaN,TIBALDI,45.44119,9.17813,Private room,100.0,4,10,2019-04-13,0.06,1,358,0,NaN
1,23986,Rental unit in Milan · ★4.64 · 1 bedroom · 1 bed · 1 bath,95941,Jeremy,NaN,NAVIGLI,45.44806,9.17373,Entire home/apt,150.0,1,26,2023-07-29,0.18,1,359,4,NaN
2,24107,Condo in Milan · ★4.50 · 1 bedroom · 6 beds · 1 bath,46951,Valeria,NaN,CITTA' STUDI,45.47179,9.23669,Entire home/apt,100.0,1,4,2013-07-31,0.02,1,365,0,NaN
3,40470,Rental unit in Milan · ★4.67 · 2 bedrooms · 4 beds · 1 bath,174203,Giacinto,NaN,VIALE MONZA,45.52023,9.22747,Entire home/apt,80.0,3,41,2023-09-09,0.26,2,290,3,NaN
4,304050,Rental unit in Milan · ★4.91 · 1 bedroom · 1 bed · 1 private bath,1566887,Elena,NaN,XXII MARZO,45.45709,9.21531,Private room,49.0,2,264,2023-11-21,1.92,2,0,33,NaN


### Cleaning the unwanted clumns from listing

In [78]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24936 entries, 0 to 24935
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              24936 non-null  int64  
 1   name                            24936 non-null  object 
 2   host_id                         24936 non-null  int64  
 3   host_name                       24936 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   24936 non-null  object 
 6   latitude                        24936 non-null  float64
 7   longitude                       24936 non-null  float64
 8   room_type                       24936 non-null  object 
 9   price                           21023 non-null  float64
 10  minimum_nights                  24936 non-null  int64  
 11  number_of_reviews               24936 non-null  int64  
 12  last_review                     

In [79]:
unwanted = ['license','availability_365','host_name','neighbourhood_group']
df_listing.drop(columns=unwanted, inplace=True)

### Keep only the listings that have at least one review in 2023

In [80]:
df_listing['last_review'] = pd.to_datetime(df_listing['last_review'], format=r'%Y-%m-%d')

In [81]:
year2023 = datetime(2023,1,1)

df_listing = df_listing[df_listing['last_review'] > year2023]

### Create a column with the star number

In [82]:
pattern = r'★([\d\.]+)'

df_listing['stars'] = df_listing['name'].str.findall(pattern).str.get(0)
df_listing.head()

,id,name,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,number_of_reviews_ltm,stars
1,23986,Rental unit in Milan · ★4.64 · 1 bedroom · 1 bed · 1 bath,95941,NAVIGLI,45.44806,9.17373,Entire home/apt,150.0,1,26,2023-07-29,0.18,1,4,4.64
3,40470,Rental unit in Milan · ★4.67 · 2 bedrooms · 4 beds · 1 bath,174203,VIALE MONZA,45.52023,9.22747,Entire home/apt,80.0,3,41,2023-09-09,0.26,2,3,4.67
4,304050,Rental unit in Milan · ★4.91 · 1 bedroom · 1 bed · 1 private bath,1566887,XXII MARZO,45.45709,9.21531,Private room,49.0,2,264,2023-11-21,1.92,2,33,4.91
5,46536,Rental unit in Milan · ★4.53 · 2 bedrooms · 3 beds · 1 bath,138683,VIALE MONZA,45.52276,9.22478,Entire home/apt,110.0,3,35,2023-10-30,0.24,1,5,4.53
12,333223,Rental unit in Milano · ★5.0 · 1 bedroom · 2 beds · 1 shared bath,1697947,MAGENTA - S. VITTORE,45.46097,9.16431,Private room,150.0,1,3,2023-04-19,0.02,5,1,5.0


In [83]:
df_listing.iloc[4,1]

'Rental unit in Milano · ★5.0 · 1 bedroom · 2 beds · 1 shared bath'

In [84]:
df_listing.iloc[2,1]

'Rental unit in Milan · ★4.91 · 1 bedroom · 1 bed · 1 private bath'

In [85]:
# Some listing have for example 1.5 beds.
# For now I get the 1.5.

pattern_bedroom = r'([\d\.]*) bedr'

df_listing['bedroom'] = df_listing['name'].str.findall(pattern_bedroom).str.get(0)

In [86]:
pattern_bed = r'([\d\.]*) beds? '

df_listing['bed'] = df_listing['name'].str.findall(pattern_bed).str.get(0)

In [87]:
# After an iterative process I found that
# there are only bath, private bath, and shared bath

pattern_bath = r'([\d\.]*)[\sprivate]*[\sshared]* bath'

df_listing['bath'] = df_listing['name'].str.findall(pattern_bath).str.get(0)

### Adjust the order and the dtypes

In [88]:
new_order = ['id','name','price','room_type','number_of_reviews','stars','bed',
             'bedroom','bath','neighbourhood','latitude', 'longitude','minimum_nights',
             'last_review','reviews_per_month','number_of_reviews_ltm',
             'calculated_host_listings_count', 'host_id',]

df_listing = df_listing[new_order]

In [89]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15259 entries, 1 to 24763
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              15259 non-null  int64         
 1   name                            15259 non-null  object        
 2   price                           14703 non-null  float64       
 3   room_type                       15259 non-null  object        
 4   number_of_reviews               15259 non-null  int64         
 5   stars                           13134 non-null  object        
 6   bed                             15070 non-null  object        
 7   bedroom                         14343 non-null  object        
 8   bath                            15229 non-null  object        
 9   neighbourhood                   15259 non-null  object        
 10  latitude                        15259 non-null  float64       
 11  longitu

In [90]:
col_to_convert = ['stars','price','number_of_reviews','bed','bedroom','bath',
                  'minimum_nights','number_of_reviews_ltm','calculated_host_listings_count',
                  'latitude','longitude','reviews_per_month']

df_listing[col_to_convert] = df_listing[col_to_convert].astype('float32')

### Handle the last null columns

In [91]:
pd.set_option('display.max_colwidth', None)

In [92]:
df_listing['id'].duplicated().any()

False

In [93]:
# Some listings don't have stars, for now I keep the null values

df_listing.loc[df_listing['stars'].isna(), 'name'][:10]

128         Vacation home in Milan · 1 bedroom · 1 bed · 1 bath
369           Rental unit in Milan · 1 bedroom · 1 bed · 1 bath
525          Rental unit in Milan · 1 bedroom · 2 beds · 1 bath
600          Rental unit in Milan · 1 bedroom · 2 beds · 1 bath
612                 Home in Milano · 1 bedroom · 1 bed · 1 bath
687     Boutique hotel in Milan · 1 bedroom · 1 bed · 1.5 baths
899             Condo in Milano · 3 bedrooms · 3 beds · 2 baths
980         Rental unit in Milano · 1 bedroom · 2 beds · 1 bath
1016              Home in Milano · 2 bedrooms · 4 beds · 1 bath
1068              Condo in Milano · 1 bedroom · 2 beds · 1 bath
Name: name, dtype: object

In [94]:
df_listing.loc[df_listing['bed'].isna(), 'name'][:10]

17             Rental unit in Milan · ★4.85 · 1 bedroom · 1 bath
211                   Loft in Milan · ★4.56 · 1 bedroom · 1 bath
707             Rental unit in Milan · ★5.0 · 1 bedroom · 1 bath
793                  Loft in Milano · ★4.54 · 1 bedroom · 1 bath
1079          Rental unit in Milano · ★4.94 · 1 bedroom · 1 bath
1150           Rental unit in Milan · ★4.77 · 1 bedroom · 1 bath
1455           Rental unit in Milan · ★4.77 · 1 bedroom · 1 bath
1851    Guesthouse in Milano · ★4.87 · 1 bedroom · 1 shared bath
2268          Rental unit in Milano · ★4.98 · 1 bedroom · 1 bath
2407           Guest suite in Milan · ★4.94 · 1 bedroom · 1 bath
Name: name, dtype: object

In [95]:
# It's really strange a home with a bedroom but with no beds.
# Since there are few rows with this problem, I just drop them.

df_listing.dropna(subset=['bed'], inplace=True)

In [96]:
# It makes sense that some homes don't have a bedroom.
# I fill the na values with 0.

df_listing.loc[df_listing['bedroom'].isna(), 'name'][:10]

22            Rental unit in Milan · ★4.54 · Studio · 1 bed · 1 bath
49                 Condo in Milan · ★4.74 · Studio · 2 beds · 1 bath
106    Serviced apartment in Milan · ★5.0 · Studio · 1 bed · 0 baths
218           Rental unit in Milan · ★4.80 · Studio · 1 bed · 1 bath
259          Rental unit in Milan · ★4.77 · Studio · 2 beds · 1 bath
377           Rental unit in Milan · ★4.79 · Studio · 1 bed · 1 bath
397          Rental unit in Milano · ★4.90 · Studio · 1 bed · 1 bath
443          Rental unit in Milano · ★4.94 · Studio · 1 bed · 1 bath
458         Rental unit in Milano · ★4.89 · Studio · 2 beds · 1 bath
473          Rental unit in Milan · ★4.75 · Studio · 2 beds · 1 bath
Name: name, dtype: object

In [97]:
df_listing['bedroom'] = df_listing['bedroom'].fillna(0)

In [98]:
# I handle the strange listings with, for example, 1.5 beds

def is_not_whole(n):

    '''This function will return a boolean,
    depending on whether n is a whole number.'''

    return n % 1 != 0

for i in ['bath','bed','bedroom']:
    print(f'Non whole rows for {i}:', df_listing[i].apply(is_not_whole).sum())

Non whole rows for bath: 547
Non whole rows for bed: 0
Non whole rows for bedroom: 0


In [99]:
# I round the number to the lower whole number

df_listing['bath'] = np.floor(df_listing['bath'])

In [100]:
# Drop the null value for price and bath

df_listing.dropna(subset=['price','bath'], inplace=True)

df_listing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14502 entries, 1 to 24763
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14502 non-null  int64         
 1   name                            14502 non-null  object        
 2   price                           14502 non-null  float32       
 3   room_type                       14502 non-null  object        
 4   number_of_reviews               14502 non-null  float32       
 5   stars                           12525 non-null  float32       
 6   bed                             14502 non-null  float32       
 7   bedroom                         14502 non-null  float32       
 8   bath                            14502 non-null  float32       
 9   neighbourhood                   14502 non-null  object        
 10  latitude                        14502 non-null  float32       
 11  longitu

### Add 'days_first_r' col 

In [101]:
df_reviews.head()

,listing_id,date
0,6400,2010-04-19
1,6400,2011-04-16
2,6400,2012-04-22
3,6400,2014-04-11
4,6400,2014-04-14


In [102]:
df_reviews['date'] = pd.to_datetime(df_reviews['date'],
                                    format='%Y-%m-%d')
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775874 entries, 0 to 775873
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   listing_id  775874 non-null  int64         
 1   date        775874 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.8 MB


In [103]:
first_revies = df_reviews.groupby('listing_id')['date'].min().to_frame().reset_index()
first_revies.head()

,listing_id,date
0,6400,2010-04-19
1,23986,2012-04-24
2,24107,2010-04-11
3,40470,2010-12-20
4,46536,2011-12-05


In [104]:
df_listing = pd.merge(df_listing, first_revies,
                        left_on='id', right_on='listing_id', how='left')
df_listing.drop(columns='listing_id', inplace=True)
df_listing.head(2)

,id,name,price,room_type,number_of_reviews,stars,bed,bedroom,bath,neighbourhood,latitude,longitude,minimum_nights,last_review,reviews_per_month,number_of_reviews_ltm,calculated_host_listings_count,host_id,date
0,23986,Rental unit in Milan · ★4.64 · 1 bedroom · 1 bed · 1 bath,150.0,Entire home/apt,26.0,4.64,1.0,1.0,1.0,NAVIGLI,45.448059,9.17373,1.0,2023-07-29,0.18,4.0,1.0,95941,2012-04-24
1,40470,Rental unit in Milan · ★4.67 · 2 bedrooms · 4 beds · 1 bath,80.0,Entire home/apt,41.0,4.67,4.0,2.0,1.0,VIALE MONZA,45.520229,9.22747,3.0,2023-09-09,0.26,3.0,2.0,174203,2010-12-20


In [106]:
df_listing['days_online'] = df_listing['last_review'].max() - df_listing['date']

df_listing.head(2)

,id,name,price,room_type,number_of_reviews,stars,bed,bedroom,bath,neighbourhood,latitude,longitude,minimum_nights,last_review,reviews_per_month,number_of_reviews_ltm,calculated_host_listings_count,host_id,date,days_online
0,23986,Rental unit in Milan · ★4.64 · 1 bedroom · 1 bed · 1 bath,150.0,Entire home/apt,26.0,4.64,1.0,1.0,1.0,NAVIGLI,45.448059,9.17373,1.0,2023-07-29,0.18,4.0,1.0,95941,2012-04-24,4261 days
1,40470,Rental unit in Milan · ★4.67 · 2 bedrooms · 4 beds · 1 bath,80.0,Entire home/apt,41.0,4.67,4.0,2.0,1.0,VIALE MONZA,45.520229,9.22747,3.0,2023-09-09,0.26,3.0,2.0,174203,2010-12-20,4752 days


In [112]:
df_listing['days_online'] = df_listing['days_online'].dt.days.astype(float)

In [114]:
df_listing.drop(columns='date', inplace=True)

df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14502 entries, 0 to 14501
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14502 non-null  int64         
 1   name                            14502 non-null  object        
 2   price                           14502 non-null  float32       
 3   room_type                       14502 non-null  object        
 4   number_of_reviews               14502 non-null  float32       
 5   stars                           12525 non-null  float32       
 6   bed                             14502 non-null  float32       
 7   bedroom                         14502 non-null  float32       
 8   bath                            14502 non-null  float32       
 9   neighbourhood                   14502 non-null  object        
 10  latitude                        14502 non-null  float32       
 11  lo

### Convert the cleaned dataset to CSV

In [115]:
df_listing.to_csv(r'C:\Users\matte\OneDrive\Desktop\GitHub\data\Airbnb\listings_cleaned.csv',
                  index=False)